In [1]:
import pandas as pd
from src import bayesian_3pt_percentage_with_credible_interval
from sqlalchemy import create_engine

In [61]:
engine = create_engine("sqlite:///Data/player_database.db")
input_data = pd.read_sql("BAYESIAN_3PT_INPUT_2023_24", engine)
common_player_info = pd.read_sql("COMMON_PLAYER_INFO", engine)
input_data = input_data.dropna()

In [70]:
players = []
player_names = []
bayesian_three_perc = []
ordinary_three_perc = []
historical_three_perc = []
lower_bounds = []
upper_bounds = []

In [71]:
for index, row in input_data.iterrows():
    if row["SEASON_3PA"] > 0:
        name_matches = common_player_info[common_player_info["PERSON_ID"] == row["PLAYER_ID"]]["DISPLAY_FIRST_LAST"].tolist()
        if len(name_matches) == 1:
            players.append(int(row["PLAYER_ID"]))
            player_names.append(name_matches[0])
            perc, lower_bound, upper_bound = bayesian_3pt_percentage_with_credible_interval(float(row["HISTORICAL_3P%"]), row["SEASON_3PA"], row["SEASON_3PM"])
            bayesian_three_perc.append(perc)
            lower_bounds.append(lower_bound)
            upper_bounds.append(upper_bound)
            ordinary_three_perc.append(row["SEASON_3PM"]/row["SEASON_3PA"])
            historical_three_perc.append(row["HISTORICAL_3P%"])

In [72]:
out_dict = {"PLAYER_ID": players, "PLAYER": player_names, "BAYESIAN_3P%": bayesian_three_perc, "LOWER_BOUND": lower_bounds, "UPPER_BOUND": upper_bounds, "SEASON_3P%": ordinary_three_perc, "HISTORICAL_3P%": historical_three_perc}
df = pd.DataFrame(out_dict)
df.to_sql("BAYESIAN_3PT_OUTPUT_2023_24", con=engine, if_exists="replace",index=False)

470

In [74]:
print(df.head())

   PLAYER_ID                    PLAYER  BAYESIAN_3P%  LOWER_BOUND  \
0    1630173          Precious Achiuwa      0.320755     0.223223   
1    1628389               Bam Adebayo      0.129731     0.035035   
2    1630534              Ochai Agbaji      0.355263     0.257257   
3    1630583              Santi Aldama      0.318471     0.220220   
4    1629638  Nickeil Alexander-Walker      0.337296     0.239239   

   UPPER_BOUND  SEASON_3P%  HISTORICAL_3P%  
0     0.418418    0.276596        0.320755  
1     0.227227    0.142857        0.129032  
2     0.453453    0.344538        0.355263  
3     0.416416    0.352000        0.318471  
4     0.435435    0.367188        0.337296  
